In [ ]:
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibration.calibrator import Localizer
from skrgbd.calibration.calibration.presets.tis import TISCalibrator

### Calculate reference camera poses w.r.t the board

In [ ]:
images_root = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/calibration/images.tabletop'
results_root = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/calibration/results/tabletop'

calib_class = TISCalibrator
cam_name = 'tis_left'
calib_dir = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/calibration/results/camera_plane/tis_left/calibration@central_generic,40px'

In [ ]:
calibrator = calib_class(img_dir=f'{images_root}/{cam_name}', results_dir=f'{results_root}/{cam_name}')
calibrator.extract_features(visualize=True)

In [ ]:
localizer = Localizer(
    [calib_dir],
    [f'{results_root}/{cam_name}/dataset.bin'],
    results_dir=f'{results_root}/{cam_name}/localization'
)
localizer.prepare_to_localize()
localizer.localize(visualize=True)

### Save poses to COLMAP

In [ ]:
from pathlib import Path

import numpy as np
import scipy.spatial
import torch

from colmap.read_write_model import Image, write_images_text

from skrgbd.calibration.calibrations.small_scale_sphere import Calibration
from skrgbd.calibration.eth_tool.ex_poses import get_poses, Poses
from skrgbd.devices.robot.robot_on_table import RobotOnTable
from skrgbd.data.io.poses import save_poses
from skrgbd.data.dataset.pathfinder import sensor_to_cam_mode
from skrgbd.calibration.trajectories.trajectory import Trajectory as TrajectoryBase

In [ ]:
class Trajectory(TrajectoryBase):
    points = None
    robot = None

    def __init__(self, robot=None):
        if robot is None:
            robot = RobotOnTable(simulation=True)
        self.robot = robot
        self.points = self.robot.generate_trajectory_points()
    

trajectory = Trajectory()

In [ ]:
ref_cam_name = 'tis_left'


poses_yaml = f'{results_root}/{ref_cam_name}/localization/rig_tr_global.yaml'
dataset_bin = f'{results_root}/{ref_cam_name}/localization/dataset.bin'
world_to_rig, pose_found = get_poses(trajectory, poses_yaml, dataset_bin)
assert pose_found.all()

rig_to_ref_cam = Poses(f'{results_root}/{ref_cam_name}/localization/camera_tr_rig.yaml')
rig_to_ref_cam = rig_to_ref_cam[0]

world_to_ref_cam = rig_to_ref_cam @ world_to_rig; del rig_to_ref_cam, world_to_rig

In [ ]:
calibration = Calibration()

In [ ]:
all_sensors = {
    'real_sense_rgb', 'real_sense_ir', 'real_sense_ir_right',
    'kinect_v2_rgb', 'kinect_v2_ir',
    'tis_left', 'tis_right',
    'phone_left_rgb', 'phone_left_ir',
    'phone_right_rgb', 'phone_right_ir'
}

world_to_cam = dict()
for sensor in all_sensors:
    if sensor == ref_cam_name:
        world_to_cam[sensor] = world_to_ref_cam
    else:
        world_to_cam[sensor] = calibration.rig_to_cam[sensor] @ calibration.rig_to_cam[ref_cam_name].inverse() @ world_to_ref_cam

In [ ]:
def save_poses(images_txt, trajectory, world_to_cam, sensor, cam_i=0):
    if sensor.startswith('phone') and sensor.endswith('rgb'):
        ext = 'jpg'
    else:
        ext = 'png'

    colmap_images = dict()
    for pos_i in range(len(trajectory)):
        pos_id = trajectory[pos_i]
        rotmat = world_to_cam[pos_i, :3, :3].numpy()
        rotmat = scipy.spatial.transform.Rotation.from_matrix(rotmat)
        xyzw = rotmat.as_quat()
        wxyz = np.roll(xyzw, 1)
        tvec = world_to_cam[pos_i, :3, 3].numpy()

        img_filename = f'{pos_id}.{ext}'
        image_id = pos_i + 1  # COLMAP's image_id is one-based

        colmap_images[pos_i] = Image(
            id=image_id, qvec=wxyz.astype(np.float32), tvec=tvec.astype(np.float32), camera_id=cam_i, name=img_filename,
            xys=[], point3D_ids=[]
        )

    return write_images_text(colmap_images, images_txt)


calib_root = '/mnt/data/sk_rgbd_data/wp2/dataset/calibration'
for sensor in all_sensors:
    cam, mode = sensor_to_cam_mode[sensor]
    images_txt = f'{calib_root}/{cam}/{mode}/images.txt'
    Path(images_txt).parent.mkdir(exist_ok=True, parents=True)
    save_poses(images_txt, trajectory, world_to_cam[sensor], sensor)